In [10]:
# install fast_ml library
! pip install fast_ml --quiet

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Get a Regressor first! 

# Explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.experimental import 

# Now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingRegressor
#from sklearn.ensemble import 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier 


from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

import itertools

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error

In [4]:
from lazypredict.Supervised import LazyClassifier, LazyRegressor

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
# evaluate a logistic regression model using k-fold cross-validation
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [11]:
from fast_ml.model_development import train_valid_test_split

## Reading the saved data 

In [6]:
# loading
full_df = pd.read_pickle('full_df_saved27Aug2023')

In [ ]:
import matplotlib.pyplot as plt# Pie chart
labels = list(full_df.columns[:5]) #['Frogs', 'Hogs', 'Dogs', 'Logs']
sizes = [sum(full_df.iloc[:,i]) for i in range(5)]# [2018, 9234, 18, 1529, 16]#[15, 30, 45, 10]#colors
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#ffb3e6']#['red','#66b3ff','#99ff99','brown']#,'pink','brown']
 
fig1,ax1 = plt.subplots(figsize=(6,6), facecolor="w", dpi=200)#1200)
#fig1, ax1 = plt.subplots()
patches, texts, autotexts = ax1.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90)
for text in texts:
    text.set_color('black')#'grey')
for autotext in autotexts:
    autotext.set_color('black')#'grey')# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [7]:
def split_dataframe(dfin, ratio):
    training_data = dfin.sample(frac=ratio, random_state=25)
    testing_data = dfin.drop(training_data.index)
    
    return training_data, testing_data

In [8]:
def one_hot(i, vec_len):
  vector = [0]*vec_len
  vector[i] = 1
  return vector

def evaluate(y,y_hat,labels):
  print(classification_report(y,y_hat))
  cm = confusion_matrix(y,y_hat)
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  cmat = pd.DataFrame(cm)
  cmat.columns = labels
  #cmat.set_index([pd.Index(labels, "")],inplace=True)
  sns.heatmap(cmat,cmap="YlGnBu", annot=True)
  plt.title("Confusion Matrix")

def format_prediction(predicted_outputs):
  predicted_image_array = np.zeros_like(image_array)
  i = 0
  for row,pixels in enumerate(predicted_image_array):
    for column in range(len(pixels)):
      p = np.argmax(predicted_outputs[i])
      predicted_image_array[row][column] = [WHITE,RED,BLUE][p]
      i += 1
  return predicted_image_array

In [15]:
training_inputs = full_df.iloc[:,5:]
training_outputs_labels = full_df.iloc[:,:5]
#xgb_classifier.fit(training_inputs, training_outputs_labels)
#xgb_predictions_labels = xgb_classifier.predict(training_inputs)

In [9]:

X = full_df.drop(columns=['Healthy','M-KOMPG','M-KOMPD','M-KOMPA','M-KOMPM'])
y = full_df[{'Healthy','M-KOMPG','M-KOMPD','M-KOMPA','M-KOMPM'}]
cat_labels = np.where(y==1)[1]
y = pd.DataFrame(cat_labels,columns=['Result'])


#M = 000
#X = X.iloc[:M]
#y = y.iloc[:M]

# Creating train test split
# Call train_test_split on the data and capture the results
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3,test_size=0.25)

# train, test, validation split
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, 
                                                                            target = 'Result', 
                                                                            train_size=0.6, 
                                                                            valid_size=0.2, 
                                                                            test_size=0.2)

# reset index
for data in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    data.reset_index(drop=True, inplace=True)

In [155]:
y_train[:10]

,Result
3264,0
34864,0
33490,0
18925,1
8534,0
8834,0
41223,0
20170,0
60978,0
17651,0


In [160]:
%%time

reg = LazyClassifier(ignore_warnings=False, custom_metric=None)

#reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
N=16500
models, predictions = reg.fit(X_train[:N], X_test[:N], y_train[:N], y_test[:N])
print(models)

  3%|█▌                                          | 1/29 [00:15<07:07, 15.27s/it]

ROC AUC couldn't be calculated for AdaBoostClassifier
multi_class must be in ('ovo', 'ovr')


  7%|███                                         | 2/29 [01:50<27:57, 62.11s/it]

ROC AUC couldn't be calculated for BaggingClassifier
multi_class must be in ('ovo', 'ovr')


 10%|████▌                                       | 3/29 [01:50<14:43, 33.98s/it]

...

ROC AUC couldn't be calculated for SVC
multi_class must be in ('ovo', 'ovr')
StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'


 97%|█████████████████████████████████████████▌ | 28/29 [14:30<00:13, 13.40s/it]

ROC AUC couldn't be calculated for XGBClassifier
multi_class must be in ('ovo', 'ovr')


100%|███████████████████████████████████████████| 29/29 [14:34<00:00, 30.16s/it]

ROC AUC couldn't be calculated for LGBMClassifier
multi_class must be in ('ovo', 'ovr')
                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
ExtraTreesClassifier               0.93               0.63    None      0.92   
RandomForestClassifier             0.93               0.63    None      0.92   
XGBClassifier                      0.93               0.63    None      0.92   
BaggingClassifier                  0.92               0.62    None      0.92   
LGBMClassifier                     0.92               0.62    None      0.92   
DecisionTreeClassifier             0.88               0.61    None      0.88   
ExtraTreeClassifier                0.87               0.56    None      0.87   
KNeighborsClassifier               0.89               0.54    None      0.88   
QuadraticDiscriminantAnalysis      0.89               0.52    None      0.89   
GaussianNB                      

In [159]:
%%time

#reg = LazyClassifier(ignore_warnings=False, custom_metric=None)

reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
N=10000
models, predictions = reg.fit(X_train[:N], X_test[:N], y_train[:N], y_test[:N])
print(models)

 21%|█████████▍                                  | 9/42 [01:50<09:42, 17.65s/it]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range for family GammaDistribution


 79%|█████████████████████████████████▊         | 33/42 [06:15<01:18,  8.70s/it]

RANSACRegressor model failed to execute
RANSAC could not find a valid consensus set. All `max_trials` iterations were skipped because each randomly chosen sub-sample failed the passing criteria. See estimator attributes for diagnostics (n_skips*).


100%|███████████████████████████████████████████| 42/42 [13:03<00:00, 18.65s/it]

                                  Adjusted R-Squared              R-Squared  \
Model                                                                         
ExtraTreesRegressor                             0.59                   0.60   
RandomForestRegressor                           0.55                   0.56   
LGBMRegressor                                   0.53                   0.54   
HistGradientBoostingRegressor                   0.52                   0.53   
BaggingRegressor                                0.49                   0.51   
XGBRegressor                                    0.45                   0.47   
MLPRegressor                                    0.43                   0.44   
GradientBoostingRegressor                       0.39                   0.41   
KNeighborsRegressor                             0.38                   0.40   
SVR                                             0.36                   0.37   
NuSVR                                           0.33

In [35]:
y_train[110:130]

,Result
30971,0
60077,0
29302,1
9438,4
30847,0
43344,0
37016,0
19194,0
32976,0
24200,2


In [16]:
from xgboost import XGBClassifier
xgb_classifier = OneVsRestClassifier(XGBClassifier())

In [17]:
xgb_classifier.fit(training_inputs, training_outputs_labels)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, reg_alpha=None,
                                      

In [18]:
training_outputs_labels_l = [np.argmax(training_outputs_labels.iloc[p,:]) for p in range(len(training_outputs_labels))]

In [ ]:
evaluate(testing_outputs_labels_l, rf_predictions_labels, labels)
xgb_predictions = [one_hot(p,z) for p in rf_predictions_labels]
Image.fromarray(format_prediction(xgb_predictions))

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Training the Random forest here
rf_classifier.fit(X_train, y_train)#training_inputs, training_outputs_labels)

RandomForestClassifier()

In [32]:
rf_predictions = rf_classifier.predict(X_test)#testing_inputs)#input_histdf.iloc[N:,:])#training_inputs)
rf_predictions_labels = [np.argmax(p) for p in rf_predictions]
y_hat = rf_predictions #_labels

In [36]:
temp = np.array(y_test)#testing_outputs_labels)#histdf.iloc[N:,:])
#y = temp
#y
#temp = np.array(histdf.iloc[:N,:])
#histdf_labels = [np.argmax(p) for p in temp]
y = temp#histdf_labels

In [37]:
confusion_matrix(y,y_hat)

array([[13060,    49,    15,     6],
       [  285,  2299,    17,    31],
       [   85,    82,   257,    24],
       [   89,   235,    24,   207]])

In [38]:
print("This is for the ")
print(classification_report(y,y_hat))

This is for the 
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     13130
           1       0.86      0.87      0.87      2632
           2       0.82      0.57      0.68       448
           4       0.77      0.37      0.50       555

    accuracy                           0.94     16765
   macro avg       0.86      0.70      0.76     16765
weighted avg       0.94      0.94      0.94     16765



In [99]:
modelETR = ExtraTreesRegressor() # 10 minutes
modelXGBM = XGBRegressor()  # 10 seconds
modelHGBR = HistGradientBoostingRegressor() # 14 seconds
modelRF = RandomForestRegressor() # 54 minutes!
modelLGBM = LGBMRegressor() # 2 seconds

In [138]:
%%time
# Takes 6 seconds
#modelLGBM = LGBMRegressor()
modelHGBR.fit(X_train, y_train)#features_train, labels_train)

CPU times: user 4min 34s, sys: 362 ms, total: 4min 35s
Wall time: 6.41 s


HistGradientBoostingRegressor()

In [62]:
%%time
# Takes 2 seconds
#modelLGBM = LGBMRegressor()
modelLGBM.fit(X_train, y_train)#features_train, labels_train)

CPU times: user 49.2 s, sys: 383 ms, total: 49.5 s
Wall time: 1.67 s


LGBMRegressor()

In [92]:
kf = KFold(n_splits =10,shuffle=True,random_state=160) # n_splits=10
feats = pd.DataFrame({'features': X_train.columns})
gbm_predictions = []
cv_score_ = 0
oof_preds = np.zeros((full_df.shape[0],))

,Result
3264,0
34864,0
33490,0
18925,1
8534,0
...,...
43883,0
11261,0
25365,0
48056,0


In [100]:
XX = X_train
yy = y_train

In [101]:
%%time

gbm_score = 0
for i,(tr_index,test_index) in enumerate(kf.split(XX,yy)):
  
  #print(tr_index)
  print(f'######### FOLD {i+1} / {kf.n_splits} ')
  
  XX_train,yy_train = XX.iloc[tr_index,:],yy.iloc[tr_index]
  #tempX = XX.loc[XX.index.intersection(tr_index),:]  
  #tempX = X.reindex[tr_index,:]
  #tempy = yy.loc[yy.index.intersection(tr_index)] 
  #XX_train, yy_train = tempX, tempy
  #tips_filtered = tips_df.reindex(columns = filtered_columns).

  XX_test,yy_test = XX.iloc[test_index,:],yy.iloc[test_index]
  #tempX = XX.loc[XX.index.intersection(test_index),:] 
  #tempy = yy.loc[yy.index.intersection(test_index)] 
  #XX_test, yy_test = tempX, tempy
  
  #gbm = xgb.ExtraTreesRegressor(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)
  #gbm = xgb.XGBRegressor(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)
  # WORKED gbm = lgb.LGBMClassifier(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)

  #gbm = xgb.LGBMRegressor(eval_metric = 'rmse',n_estimators = 2000,learning_rate = 0.01,seed=162,random_state = 162,colsample_bytree=0.65)
  # WORKED with below fit gbm = ExtraTreesRegressor(n_estimators = 2000,random_state = 162)
  # WORKED with above gbm.fit(X_train, y_train)
  #gbm = #ExtraTreesClassifier(n_estimators=10, random_state=42)
  gbm=modelHGBR# parallel CV=0.17 2min for 10 folds
  #gbm=modelXGBM# Parallel. CV = 0.35 2 min for 10 folds
  #gbm=modelLGBM# Parallel. CV = 0.52 0.5 min for 10 folds
  #gbm=LGBMRegressor()
  #gbm=modelETR# single thread. CV = 0.29 15 min for 2 folds
  print(len(XX_train), len(yy_train))
  gbm.fit(XX_train,yy_train)

  #gbm.fit(X_train,y_train,eval_set = [(X_test, y_test)],early_stopping_rounds  = 200,verbose=100)
  
  #gbm.fit(X_train,y_train,eval_set = [(X_test, y_test)],early_stopping_rounds  = 200,verbose=100)
  
  gbm_score += gbm.score(XX_test, yy_test)/ kf.n_splits
  cv_score_ += mean_squared_error(yy_test, gbm.predict(XX_test), squared=False) / kf.n_splits
  #oof_preds[test_index] = gbm.predict(X_test)
  
  #preds = gbm.predict(nnfielddfNC[X_train.columns])
  #gbm_predictions.append(preds)
  print('iter: ',i,'CV RMSE : ', cv_score_)
    
  #feats[f'Fold {i}'] = gbm.feature_importances_
#feats['Importances'] = feats.mean(axis=1)
print('CV RMSE : ', cv_score_)
#preds_xgb = np.average(gbm_predictions, axis=0)
#print(preds_xgb.shape)

######### FOLD 1 / 10 
45264 45264
iter:  0 CV RMSE :  0.5753477279367962
######### FOLD 2 / 10 
45264 45264
iter:  1 CV RMSE :  0.6277312889630838
######### FOLD 3 / 10 
45264 45264
iter:  2 CV RMSE :  0.678916001081683
######### FOLD 4 / 10 
45264 45264
iter:  3 CV RMSE :  0.7327823304628934
######### FOLD 5 / 10 
45265 45265
iter:  4 CV RMSE :  0.7867908897191747
######### FOLD 6 / 10 
45265 45265
iter:  5 CV RMSE :  0.8379145364082294
######### FOLD 7 / 10 
45265 45265
iter:  6 CV RMSE :  0.8871166672009297
######### FOLD 8 / 10 
45265 45265
iter:  7 CV RMSE :  0.9416863139881468
######### FOLD 9 / 10 
45265 45265
iter:  8 CV RMSE :  0.993615949036049
######### FOLD 10 / 10 
45265 45265
iter:  9 CV RMSE :  1.0483813124478725
CV RMSE :  1.0483813124478725
CPU times: user 1h 17min 34s, sys: 4min 12s, total: 1h 21min 47s
Wall time: 1min 43s


In [102]:
feats['Importances'] = feats.mean(axis=1)
print( ' CV RMSE : ',cv_score_)
preds_xgb = np.average(gbm_predictions, axis=0)
print(preds_xgb.shape)
print("CV Accuracy is :", gbm_score)

 CV RMSE :  1.0483813124478725
()
CV Accuracy is : 0.5903673159443066


In [119]:
predres = list(gbm.predict(X_test.iloc[:100,:]))
trueres = list(y_test['Result'].iloc[:100])

In [134]:
np.round(predres[:30])

array([ 0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        3.,  0.,  0.,  0.])

In [135]:
trueres.iloc[:30]

61623    0
57263    0
41088    0
16056    1
15632    4
38069    0
48628    0
40268    0
66414    0
35555    0
41800    0
47294    0
20588    0
38445    4
8333     0
6059     0
63636    0
37602    0
23961    1
45095    0
61480    0
60020    0
35453    0
2283     1
25667    0
11847    1
24297    4
57083    0
34853    0
47236    0
Name: Result, dtype: int64

In [142]:
#row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
#k=17686
sumcorrect=0
sumincorrect=0
eps=0.01
N=len(y_test)
predres = modelHGBR.predict(X_test.iloc[:N,:])
trueres = y_test['Result'].iloc[:N]
for k in range(N):#range(16797):
    #row=X_test.iloc[k,:]
    #trueres[k]#=y_test.iloc[k]
    #predres=gbm.predict(row)
    #print("true", trueres, "pred", predres)
    if abs(trueres.iloc[k]-np.round(predres[k]))<eps:
        sumcorrect+=1 
    else:
        sumincorrect+=1
print("Correct: ",sumcorrect,". Incorrect: ",sumincorrect)        
#print(y_train.iloc[k])

Correct:  14891 . Incorrect:  1874


In [143]:
print("missclassification rate = ", (sumincorrect)/sumcorrect)

missclassification rate =  0.12584782754684037


In [146]:
y=[]
y_hat = []
for k in trueres:
    y.append(k)
for k in predres:
    y_hat.append(np.round(k))

In [147]:
print("This is for the ")
print(classification_report(y, y_hat))

This is for the 
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97     13130
         1.0       0.80      0.66      0.72      2632
         2.0       0.23      0.47      0.31       448
         3.0       0.00      0.00      0.00         0
         4.0       0.86      0.03      0.07       555

    accuracy                           0.89     16765
   macro avg       0.57      0.43      0.41     16765
weighted avg       0.91      0.89      0.89     16765



In [148]:
confusion_matrix(y,y_hat)

array([[12918,   191,    18,     3,     0],
       [  412,  1742,   450,    26,     2],
       [   60,   142,   212,    33,     1],
       [    0,     0,     0,     0,     0],
       [   63,   115,   230,   128,    19]])

In [ ]:
run separately
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

model_lr = LogisticRegression().fit(X_train, y_train)
probs_lr = model_lr.predict_proba(X_test)[:, 1]

model_dt = DecisionTreeClassifier().fit(X_train, y_train)
probs_dt = model_dt.predict_proba(X_test)[:, 1]

model_rf = RandomForestClassifier().fit(X_train, y_train)
probs_rf = model_rf.predict_proba(X_test)[:, 1]

model_xg = XGBClassifier().fit(X_train, y_train)
probs_xg = model_xg.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import auc, precision_recall_curve

y_test_int = y_test#.replace({'Good': 1, 'Bad': 0})

baseline_model = sum(y_test_int == 1) / len(y_test_int)

precision_lr, recall_lr, _ = precision_recall_curve(y_test_int, probs_lr)
auc_lr = auc(recall_lr, precision_lr)

precision_dt, recall_dt, _ = precision_recall_curve(y_test_int, probs_dt)
auc_dt = auc(recall_dt, precision_dt)

precision_rf, recall_rf, _ = precision_recall_curve(y_test_int, probs_rf)
auc_rf = auc(recall_rf, precision_rf)

precision_xg, recall_xg, _ = precision_recall_curve(y_test_int, probs_xg)
auc_xg = auc(recall_xg, precision_xg)


In [ ]:
plt.figure(figsize=(12, 7))
plt.plot([0, 1], [baseline_model, baseline_model], linestyle='--', label='Baseline model')
plt.plot(recall_lr, precision_lr, label=f'AUC (Logistic Regression) = {auc_lr:.2f}')
plt.plot(recall_dt, precision_dt, label=f'AUC (Decision Tree) = {auc_dt:.2f}')
plt.plot(recall_rf, precision_rf, label=f'AUC (Random Forests) = {auc_rf:.2f}')
plt.plot(recall_xg, precision_xg, label=f'AUC (XGBoost) = {auc_xg:.2f}')
plt.title('Precision-Recall Curve', size=20)
plt.xlabel('Recall', size=14)
plt.ylabel('Precision', size=14)
plt.legend();